# Data Processing and Analytics

This notebook will show you the workflow of a typical data analytic job in pandas and teach you how to make GIF for timeseries data, and making map plots using an Xarray like plot library.

In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the folder
# which was created in your username directory under /scratch/vp91

#TODO 
os.chdir(os.path.expandvars("/scratch/vp91/$USER/Data-Analytics/"))


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import xarray as xr
import matplotlib.animation as animation 
import os
import iris
import matplotlib.dates as mdates
import iris.plot as iplt

# About the Dataset  
## Context  
The California Cooperative Oceanic Fisheries Investigations (CalCOFI) are a unique partnership of the California Department of Fish & Wildlife, NOAA Fisheries Service and Scripps Institution of Oceanography. The organization was formed in 1949 to study the ecological aspects of the sardine population collapse off California.  

The CalCOFI data set represents the longest (1949-present) and most complete (more than 50,000 sampling stations) time series of oceanographic and larval fish data in the world. It includes abundance data on the larvae of over 250 species of fish; larval length frequency data and egg abundance data on key commercial species; and oceanographic and plankton data. The physical, chemical, and biological data collected at regular time and space intervals quickly became valuable for documenting climatic cycles in the California Current and a range of biological responses to them.  

Data collected at depths down to 500 m include: temperature, salinity, oxygen, phosphate, silicate, nitrate and nitrite, chlorophyll, transmissometer, PAR, C14 primary productivity, phytoplankton biodiversity, zooplankton biomass, and zooplankton biodiversity.

In [ ]:
dataframe_raw = pd.read_csv('/scratch/vp91/zxw900/AAPP_pytorch/CalCOFI_bottle.csv')
dataframe_raw.head()

# Pandas - Data Exploration And Anlysis

## Data Overview

From the dateframe above, we can see this dataframe has 74 columns! And we can already see a lot of NaN values. Let's have a look how many NA values for each column:

In [ ]:
dataframe_raw.info()

So not all columns are useful, especially those with mostly empty values. To have a look at numerical value distributions, we can also call the `DataFrame.describe()` function in pandas:

In [ ]:
dataframe_raw.describe()

## Narrow Down Your Focus

Let's select first a few columns based on the column value description above and less null values. Some column descriptions as below:  
- `Depthm` which indicates the datacollection bottle depth in the sea
- `T_degC` which records the water temperature
- `Salnty` which is the Salinity in g of salt per kg of water (g/kg)
- `O2ml_L` which represents O2 mixing ration in ml/L  
- `O2Sat` which is the Oxygen sacturation level


In [ ]:
calcofi_subset = dataframe_raw.loc[:,'Depthm':'O2Satq']

In [ ]:
#let's get the column names
input_cols = list(calcofi_subset.columns.values)
print(input_cols)

Now let's use the correlation function to see if any of these columns are showing correlated features: 

In [ ]:
# Compute the correlation matrix
# corr = calcofi_subset.drop(['T_degC'], axis=1).corr()
corr = calcofi_subset.corr()

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(15, 15))
cmap = sns.diverging_palette(200, 10, as_cmap=True)
sns.heatmap(corr, cmap=cmap, annot=True, cbar_kws={"shrink": .5});

From the heatmap we can observe many correlated columns through the red highlights. Now if we want to know what effects the sea water temperture, we can easily identify columns like O2ml_L, O2Sat, Oxy_µmol/Kg, etc. But there are also some columns on the right hand side looks like having negative correlations with T_degC values. Let's plot the correlation values for the water temperature and have a more camparision.

In [ ]:
corr_with_temp = calcofi_subset.corr()['T_degC'].sort_values(ascending=False)
plt.figure(figsize=(10,10))
corr_with_temp.drop('T_degC').plot.bar()
plt.show();

## Dealing with NA Values

Now we single out above columns with water temperature and examine the data in detail to see how much null values they have and how can we deal with it.

In [ ]:
dataframe = calcofi_subset.filter(items=['T_degC', 'Depthm', 'Salnty', 'O2ml_L', 'STheta', 'O2Sat', 'Oxy_µmol/Kg'])
# count null values in dataframe
dataframe.isnull().sum()

How to approach null values depends on your downstream tasks and the original data distribution. Idealy data would be normally distributed like a bell shaped histogram. So here we compare 2 methods of dealing NA values, dropping all na values and padding them with mean values.

In [ ]:
# Let's compare for each variable options fillna() vs mean()
fig, ax=plt.subplots(1,2)
sns.histplot(dataframe.T_degC.dropna(), ax=ax[0])
ax[0].set_title("Dropna")
sns.histplot(dataframe.T_degC.fillna(dataframe.T_degC.mean()), ax=ax[1])
ax[1].set_title("Fillna with mean")

fig, ax=plt.subplots(1,2)
sns.histplot(dataframe.Salnty.dropna(), ax=ax[0])
ax[0].set_title("Dropna")
sns.histplot(dataframe.Salnty.fillna(dataframe.Salnty.mean()), ax=ax[1])
ax[1].set_title("Fillna with mean")

fig, ax=plt.subplots(1,2)
sns.histplot(dataframe.O2ml_L.dropna(), ax=ax[0])
ax[0].set_title("Dropna")
sns.histplot(dataframe.O2ml_L.fillna(dataframe.O2ml_L.mean()), ax=ax[1])
ax[1].set_title("Fillna with mean")

fig, ax=plt.subplots(1,2)
sns.histplot(dataframe.STheta.dropna(), ax=ax[0])
ax[0].set_title("Dropna")
sns.histplot(dataframe.STheta.fillna(dataframe.STheta.mean()), ax=ax[1])
ax[1].set_title("Fillna with mean")

fig, ax=plt.subplots(1,2)
sns.histplot(dataframe.O2Sat.dropna(), ax=ax[0])
ax[0].set_title("Dropna")
sns.histplot(dataframe.O2Sat.fillna(dataframe.O2Sat.mean()), ax=ax[1])
ax[1].set_title("Fillna with mean")

fig, ax=plt.subplots(1,2)
sns.histplot(dataframe['Oxy_µmol/Kg'].dropna(), ax=ax[0])
ax[0].set_title("Dropna")
sns.histplot(dataframe['Oxy_µmol/Kg'].fillna(dataframe['Oxy_µmol/Kg'].mean()), ax=ax[1])
ax[1].set_title("Fillna with mean")

We usually would want the data to be closer to normalised distribution, which is higher counts towards the centre of the graph. So let's try fill NA values with mean values so we don't lose too much data.

In [ ]:
def customize_dataset(data):
    dataframe = data.copy(deep=True)
    #select only 7 variables
    dataframe = dataframe.filter(items=['T_degC', 'Depthm', 'Salnty', 'O2ml_L', 'STheta', 'O2Sat', 'Oxy_µmol/Kg'])
    #fill na values with mean
    dataframe = dataframe.groupby(dataframe.columns, axis = 1).transform(lambda x: x.fillna(x.mean()))
    return dataframe

In [ ]:
df = customize_dataset(calcofi_subset)
# verify that processed dataframe has no NA values now.
df.isnull().sum() 

Now remember a correlation does not necessarily mean it is a linear relationship. We can plot the scatter plots to see the more detailed distribution relationships between each column and water temperature.

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(20,15))
#fig 1
sns.scatterplot(x=df['Depthm'], y=df['T_degC'],ax=ax[0][0],color='#39ac73')
ax[0,0].set_title('relation Between Depthm and T_degC ')
#fig 2
sns.scatterplot(x=df['Salnty'], y=df['T_degC'], color='#3385ff',ax=ax[0][1])
ax[0][1].set_title('relation Between Salnty and T_degC ')
#fig 3
sns.scatterplot(x=df['O2ml_L'], y=df['T_degC'], color='#ff5c33',ax=ax[0][2])
ax[0][2].set_title('relation Between O2ml_L and T_degC ')
#fig 4
sns.scatterplot(x=df['STheta'],y=df['T_degC'], color='#cccc00',ax=ax[1][0])
ax[1][0].set_title('relation Between STheta and T_degC ')
#fig 5
sns.scatterplot(x=df['O2Sat'],y=df['T_degC'], color='#cccc00',ax=ax[1][1])
ax[1][1].set_title('relation Between O2Sat and T_degC ')
#fig 6
sns.scatterplot(x=df['Oxy_µmol/Kg'],y=df['T_degC'], color='#cccc00',ax=ax[1][2])
ax[1][2].set_title('relation Between Oxy_µmol/Kg and T_degC ')
plt.show()

However if we only focus on the oxygen level columns, the linear relationship is very obvioius.

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,4))
#fig 1
sns.scatterplot(x=dataframe['O2Sat'], y=dataframe['O2ml_L'],ax=ax[0],color='#39ac73')
ax[0].set_title('relation Between O2Sat and O2ml_L ')
#fig 2
sns.scatterplot(x=dataframe['O2ml_L'], y=dataframe['Oxy_µmol/Kg'],ax=ax[1],color='#39ac73')
ax[1].set_title('relation Between O2ml_L and Oxy_µmol/Kg ')


# Xarry - Making a GIF For Sea Surface Temperature Temporal Plots

In [ ]:
### define data file of interest and open using xarray
data_path = "/scratch/vp91/zxw900/sea_surf_temp_data/*.nc"
f = xr.open_mfdataset(data_path)

In [ ]:
f

In [ ]:
# Pick your favourite location in the ocean.
# We choose somewhere in the Bass Strait 
timeseries = f.sst.sel(lon=145.0,lat=-39.0,method='nearest')

In [ ]:
x = range(1, len(timeseries.time)+1)
y = timeseries.values

Now, we can plot the the temporal temperature changes at the location we chose.

In [ ]:
### set up figure
plt.style.use('dark_background')
fig = plt.figure() 
ax = plt.axes(xlim=(1, len(x)), ylim=(min(y)-5,max(y)+5)) 
line, = ax.plot([], [], lw=2) 

### define initialisation function 
def init(): 
    ### creating an empty plot/frame 
    line.set_data([], []) 
    return line, 

### define lists to store x and y axis points 
xdata, ydata = [], [] 

### define animation function 
def animate(i): 

    ### x and y values to be plotted 
    x1 = x[i] 
    y1 = y[i] 

   ### appending new points to x1, y1 axiss points list 
    xdata.append(x1) 
    ydata.append(y1) 
    line.set_data(xdata, ydata) 
    return line, 

### setting a title for the plot 
plt.title('Sea Surface Temperature [1960 - 2016]') 
### defining the axis details 
plt.axis('on') 
plt.ylabel('Temperature (C)')
plt.xlabel('year')
plt.xticks([0,130,260,390, 520, 650], 
           ['1960', '1970', '1980','1990','2000','2010'])

### call the animator	 
anim = animation.FuncAnimation(fig, animate, init_func=init, 
							frames=len(x), interval=20, blit=True) 

### create output directory
outdir = './output'
if not os.path.exists(outdir):
    os.mkdir(outdir)

### save the animation as a GIF file 
animation_file = './output/timeseries.gif'

anim.save(animation_file,writer='imagemagick')


After saving the GIF file, run this cell to play it here:
![SegmentLocal](./output/timeseries.gif "segment")


In [ ]:
# TODO -  We can see a seasonal change of sst value in the GIF, but how about the average temperature per year?
# Changing above code to plot the change of average sea surface temperature over the years

# Xarray - Sea Surface Temperature Map Plots

`Iris` is one of the most popular python library for scientific plots with geogrpahic data. It can read netCDF directly like Xarray and keep the metadata field and use them for plotting! It gives you a powerful, format-agnostic interface for working with your data. It excels when working with multi-dimensional Earth Science data, where tabular representations become unwieldy and inefficient.  

A number of file formats are recognised by Iris, including CF-compliant NetCDF, GRIB, and PP, and it has a plugin architecture to allow other formats to be added seamlessly.

Building upon NumPy and dask, Iris scales from efficient single-machine workflows right through to multi-core clusters and HPC. Interoperability with packages from the wider scientific Python ecosystem comes from Iris’ use of standard NumPy/dask arrays as its underlying data storage.

We load the same data files like below, you can expand each row to see the Iris description of the data as well. 

In [ ]:
cubes = iris.load(data_path, 'sea_surface_temperature')
cubes

The data cube is the object to store `.nc` dataset in Iris. You can "filter" a region by accessing lat and lon values and get a CubeList.

In [ ]:
Australia = cubes.extract(iris.Constraint(latitude=lambda v: -50 < v < -10, 
                                          longitude=lambda vv: 110 < vv < 155))

Globe = cubes.extract(iris.Constraint(latitude=lambda v: -90 <= v <= 90, 
                                      longitude=lambda vv: 0 <= vv <= 360))

In [ ]:
type(Australia)

Since Iris using the similar data representation like Xarry, you can easily compute the mean over a year as well. Since we have extracted CubeList from above cells, Iris can `collapse` data in each element of the list, in our case within a year, and apply `iris.analysis.MEAN` function over it. 

In [ ]:
# some graphic design stuff
# Create a wider than normal figure to support our two plots
plt.figure(figsize=(12, 8), dpi=100)
# Manually adjust the spacings which are used when creating subplots
plt.subplots_adjust(hspace=0.05, wspace=0.3, top=0.95, bottom=0.05, left=0.075, right=0.925)

 
year = 2016
i = year-1960
# calculate the mean of a variable for our regions in Iris
cube_aus = Australia[i].collapsed('time', iris.analysis.MEAN)
cube_globe = Globe[i].collapsed('time', iris.analysis.MEAN)

# Plot for Australia
plt.subplot(1, 2, 1) # 1 row, 2 columns, 1st plot
cf_aus = iplt.contourf(cube_aus, 20)

# more graphic design stuff
plt.gca().coastlines()
colorbar_aus = plt.colorbar(cf_aus, orientation='horizontal')
plt.title('Australia ' + str(year))

# Plot for Globe
plt.subplot(1, 2, 2) # 1 row, 2 columns, 2nd plot
cf_globe = iplt.contourf(cube_globe, 20)

# more graphic design stuff
plt.gca().coastlines()
colorbar_globe = plt.colorbar(cf_globe, orientation='horizontal')
plt.title('Globe ' + str(year))

# Overall title
plt.suptitle('Sea Surface Temperature', y=1)

plt.show()

In [ ]:
# TODO - Modify a few lines in above cell to compare the mean temperatures in Global region in year 1950 and in year 2016.